In [16]:
import spacy
from spacy import displacy
from spacy.training.example import Example
from spacy.training.example import offsets_to_biluo_tags
from langdetect import detect
import pandas as pd
from spacy.util import minibatch, compounding
from unidecode import unidecode

nlp = spacy.load("fr_core_news_lg")


In [17]:
# Les phrases d'entrée
input_data = [
"1,je voudrais aller de Toulouse à bordeaux",
"2,Comment me rendre à Port-Boulet depuis la gare de Tours ?",
"3,Je veux aller voir mon ami Albert à Tours en partant de Bordeaux",
"4,Il y a-t-il des trains de Nantes à Montaigu",
"5,Une phrase sans origine ni destination",
"Si pas de numéro de séquence, on considère que c'est zéro",
"7, is there any train going from Paris to Marseille ?"
]

In [67]:
# Liste des mots pour reconnaître la destination en première
destination_keywords = [
    "à",
    "vers",
    "jusqu'à",
    "pour",
    "en direction de",
    "jusqu'à",
    # ... (ajoutez d'autres expressions si nécessaire)
] 

# Liste des mots pour reconnaître la departure en première
departure_keywords = [
    "depuis",
    "de",
    "au départ de",
    "en partant de",
    "depuis",
    "au départ de chez",
    # ... (ajoutez d'autres expressions si nécessaire)
]
# Liste des mots pour reconnaître la departure en première
passage_keywords = [
    "passant par",
    "via",
    "par le chemin de",
    "par la route de",
    "par la voie de",
    "en faisant un détour par",
    "par l'intermédiaire de",
    "en incluant",
    "avec un arrêt à",
    "parmi les étapes à",
    "en traversant",
    "en faisant escale à",
    "tout en visitant",
    "parmi les destinations à",
    "et en découvrant",
    "tout en passant par",
    "tout en explorant",
    "avec un passage à",
    "en chemin vers",
    "en voyage vers",
    # Ajoutez d'autres expressions si nécessaire
]

# Liste des mots a exclure
exclude_words = [
    "gare",
    "Gare",
]

In [19]:
# Fonction pour détecter la langue de la phrase
def detect_language(text):
    try:
        language = detect(text)
        if language != 'fr':   
            return True  # Vérifie si la langue détectée n'est pas le français
    except:
        return False

In [68]:
# Fonction pour extraire les informations pertinentes
def extract_trip_info(text):

    for mot in exclude_words:
        text = text.replace(mot, '')

    doc = nlp(text)

    locs = []
    result = []
    
    is_not_french = detect_language(text)
    if is_not_french:
        locs.append("NOT_FRENCH")
        return locs
 
    for token in doc:
        if token.ent_type_ == "LOC":
            locs.append(token.text)
    print(locs)
    if not locs:  # Si la liste est vide
            result.append('NOT_TRIP')

    # Normalisation des mots-clés des destination
    destination_keywords_normalized = [unidecode(keyword) for keyword in destination_keywords]

    # Normalisation des mots-clés des depart
    departure_keywords_normalized = [unidecode(keyword) for keyword in departure_keywords]

   # Normalisation des mots-clés des passage
    passage_keywords_normalized = [unidecode(keyword) for keyword in passage_keywords]


    departure = None
    destination = None
    passage = None
    departure_count = 0
    destination_count = 0
    passage_count = 0



    for i, token in enumerate(doc):
        # use unicode for text
        text_unicode = token.text
        if token.text in departure_keywords or text_unicode in departure_keywords or token.text in departure_keywords_normalized or text_unicode in departure_keywords_normalized and i < len(doc) - 1:
            if doc[i + 1].text in locs:
                departure = doc[i+1].text
                destination_count += 1  
    for i, token in enumerate(doc):
        # use unicode for text
        text_unicode = unidecode(token.text)
        if token.text in destination_keywords or text_unicode in destination_keywords or token.text in destination_keywords_normalized or text_unicode in destination_keywords_normalized and i > 0:
            if doc[i + 1].text in locs:
                destination = doc[i+1].text
                departure_count += 1

    # for i, token in enumerate(doc):
    #     # use unicode for text
    #     text_unicode = unidecode(token.text)
    #     print(token.text)
    #     if token.text in passage_keywords or text_unicode in passage_keywords or token.text in passage_keywords_normalized or text_unicode in passage_keywords_normalized and i < len(doc) - 1:
    #         if doc[i + 1].text in locs:
    #             passage = doc[i+1].text
    #             passage_count += 1  


    for keyword in passage_keywords:
        keyword_words = keyword.split()  # Divise l'expression-clé en mots
        keyword_length = len(keyword_words)

        for i in range(len(doc) - keyword_length + 1):
            window = [token.text for token in doc[i:i+keyword_length]]

            if window == keyword_words:
                print(window, keyword)
                passage_start = i + keyword_length
                passage_end = passage_start + 1
                if doc[passage_start:passage_end].text in locs:
                    passage = doc[passage_start:passage_end].text
                    passage_count += 1 
                    break

    if departure and destination and departure_count == 1 and destination_count == 1 and passage_count == 0 :
        result = {"DEPARTURE": departure, "DESTINATION": destination}
    elif departure and destination and passage and departure_count == 1 and destination_count == 1 and passage_count == 1:
        result = {"DEPARTURE": departure,"BETWEEN" : passage, "DESTINATION": destination}
    if (departure_count > 1) or (destination_count > 1):
        result = {'NOT_TRIP'}
        
    return result
        

In [70]:


print(extract_trip_info("Je voudrais aller de Toulouse à Paris en passant par Limoges"))

['Toulouse', 'Paris', 'Limoges']
['passant', 'par'] passant par
1
1
1
{'DEPARTURE': 'Toulouse', 'BETWEEN': 'Limoges', 'DESTINATION': 'Paris'}


In [49]:
text = "On bouge de Bordeaux pour Lyon ce soir, c'est ok pour toi ?"

print(text.find("Bordeaux"))
print((text.find("Bordeaux")+ len("Bordeaux")))

print(text.find("Lyon"))
print((text.find("Lyon")+ len("Lyon")))

12
20
26
30
